In [1]:
import torch
from torch import nn as nn
from torch import tensor

class BinaryModel(nn.Module):
    def __init__(self):
        super (BinaryModel, self).__init__() #super(BinaryModel, self).__init__() calls the constructor of the parent class nn.Module.
        self.linear1= nn.Linear(9,7)
        self.linear2= nn.Linear(7,4)
        self.linear3= nn.Linear(4,1)
        self.activation= nn.Sigmoid()

    def forward(self, x):
        x= self.linear1(x)
        x= self.linear2(x)
        x= self.linear3(x)
        x= self.activation(x)
        return x 
model = BinaryModel()


In [2]:
import pandas as pd

data = pd.read_csv('machine_failure.csv')
print(data.isnull)
print(data.isnull().sum())
data.fillna(data.mean(), inplace=True)

<bound method DataFrame.isnull of      footfall  tempMode  AQ  USS  CS  VOC  RP  IP  Temperature  fail
0           0         7   7    1   6    6  36   3            1     1
1         190         1   3    3   5    1  20   4            1     0
2          31         7   2    2   6    1  24   6            1     0
3          83         4   3    4   5    1  28   6            1     0
4         640         7   5    6   4    0  68   6            1     0
..        ...       ...  ..  ...  ..  ...  ..  ..          ...   ...
939         0         7   7    1   6    4  73   6           24     1
940         0         7   5    2   6    6  50   6           24     1
941         0         3   6    2   7    5  43   6           24     1
942         0         6   6    2   5    6  46   7           24     1
943        18         7   4    2   6    3  61   7           24     1

[944 rows x 10 columns]>
footfall       0
tempMode       0
AQ             0
USS            0
CS             0
VOC            0
RP        

In [4]:
features = data.drop('fail', axis=1)
labels = data['fail']


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


In [10]:
from sklearn.model_selection import train_test_split

# batches = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)
# batche1_trianing = batches[0]
# batche1_testing = batches[1]
# batche2_trianing = batches[2]
# batche2_testing = batches[3]
trianing_features, testing_features, trianing_labels, testing_labels = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)

trianing_features = tensor(trianing_features, dtype=torch.float32)
trianing_labels = tensor(trianing_labels.values, dtype=torch.float32)
testing_features = tensor(testing_features, dtype=torch.float32)
testing_labels = tensor(testing_labels.values, dtype=torch.float32)




# prediction = model(trianing1_tensor)
# print(prediction)

In [11]:
from torch.utils.data import DataLoader, TensorDataset

training_dataset = TensorDataset(trianing_features, trianing_labels)
testing_dataset = TensorDataset(testing_features, testing_labels)

training_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)
testing_loader = DataLoader(testing_dataset, batch_size=32, shuffle=False)


In [22]:
from torch import optim
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

model.train()

nums_of_epoch = 2000
for epoch in range(nums_of_epoch):
    for features, labels in training_loader:
        optimizer.zero_grad()
        output = model(features)
        loss  = criterion(output, labels.view(-1, 1))
        loss.backward() 
        optimizer.step()
    if epoch%100 == 0:
        print(f'Epoch {epoch+1}/{nums_of_epoch}, Loss: {loss.item()}')

Epoch 1/2000, Loss: 0.2234896421432495
Epoch 101/2000, Loss: 0.06176023185253143
Epoch 201/2000, Loss: 0.08240002393722534
Epoch 301/2000, Loss: 0.10628100484609604
Epoch 401/2000, Loss: 0.42354798316955566
Epoch 501/2000, Loss: 0.2691877782344818
Epoch 601/2000, Loss: 0.31145355105400085
Epoch 701/2000, Loss: 0.20320120453834534
Epoch 801/2000, Loss: 0.049940984696149826
Epoch 901/2000, Loss: 0.35202890634536743
Epoch 1001/2000, Loss: 0.183867409825325
Epoch 1101/2000, Loss: 0.072151780128479
Epoch 1201/2000, Loss: 0.16355229914188385
Epoch 1301/2000, Loss: 0.27561742067337036
Epoch 1401/2000, Loss: 0.19716353714466095
Epoch 1501/2000, Loss: 0.0989118218421936
Epoch 1601/2000, Loss: 0.3229464888572693
Epoch 1701/2000, Loss: 0.17537592351436615
Epoch 1801/2000, Loss: 0.4562422037124634
Epoch 1901/2000, Loss: 0.3862306773662567


In [21]:
from torchmetrics import Accuracy

accuracy = Accuracy(task='binary')
model.eval()
with torch.no_grad():
    for features, labels in testing_loader:
        output = model(features)
        prediction = output.round()
        accuracy.update(prediction, labels.view(-1, 1))
    model.train()
    print(f'Accuracy: {accuracy.compute().item()}')


Accuracy: 0.8677248954772949
